Uses [openrouteservice API](https://openrouteservice.org/dev/#/api-docs/v2/isochrones)



In [1]:
import openrouteservice as ors
import folium

# Sets up API key and client for isochrone use
ors_key = '5b3ce3597851110001cf62481647f513d9fc472d8593d2edb67288d5'
client = ors.Client(key=ors_key)

In [2]:
def generate_isochrone(start, time=300, transport="driving-car"):
    """
    Accepts a list of starting lattitude and longitude, an integer time
    in seconds (defaults to 300 seconds/5 minutes) determining the invervals
    of time to be shown on the isochrone, and a transport type which defaults 
    to a driving car.
    
    Acceptable transport values are: driving-car, foot-walking, cycling-regular,
    cycling-road, cycling-mountain, and cycling-electric
    
    Returns an isochrone
    """
    
    # Get latitude and longitude from starting coordinates 
    lat, lon = start[0], start[1]
    start = [[lon, lat]]
    
    # Creates time intervals to show on the isochrone
    time_intervals = []
    interval = 0
    for i in range(3):
        interval += time
        time_intervals.append(interval)
        
    # Builds isocrhone
    isochrone = client.isochrones(locations=start, range_type='time',range=time_intervals, profile=transport)
    map_isochrone = folium.Map(location=[lat, lon], tiles='cartodbpositron', zoom_start=15)
    
    # GeoData from the isochrone
    folium.GeoJson(isochrone, name='isochrone').add_to(map_isochrone)
    
    interval_range = []
    for k in range(len(isochrone['features'])):
        interval = isochrone['features'][k]['properties']['value']/60
        interval_range.append(interval)

    # Places marker on the starting coordinates indicating the time range
    popup_message = f'outline shows areas {transport} within {interval_range} minutes'
    folium.Marker([lat, lon], popup=popup_message, tooltip='click').add_to(map_isochrone)

    
    # return map 
    return map_isochrone    

In [3]:
"""
Test code below returns an isochrone with starting coordinates of
University of Washington, with intervals of 5 minutes, and a transport
type of driving. 

Acceptable transport values are: driving-car, foot-walking, cycling-regular,
cycling-road, cycling-mountain, and cycling-electric
"""

# Univeristy of Washington passed in Latitude Longitude 
start = [47.655548, -122.303200]
# Time passed in seconds 
time = 300
# Mode of transport
transport = "foot-walking"

generate_isochrone(start, time, transport)



In [8]:
def generate_isochrone_dest(start, end, time=300, transport="driving-car"):
    """
    Accepts a list of starting lattitude and longitude, 
    a list of ending lattide and longitude, and shows the 
    route between the two, an integer time
    in seconds (defaults to 300 seconds/5 minutes) determining the invervals
    of time to be shown on the isochrone, and a transport type which defaults 
    to a driving car.
    
    Acceptable transport values are: driving-car, foot-walking, cycling-regular,
    cycling-road, cycling-mountain, and cycling-electric
    
    Returns an isochrone showing time between two destinations.
    """
    
    # Get latitude and longitude from starting coordinates 
    lat_start, lon_start = start[0], start[1]
    start = [lon_start, lat_start]
    
    lat_end, lon_end = end[0], end[1]
    end = [lon_end, lat_end]
    
    to_travel = [start, end]
    
    # Creates time intervals to show on the isochrone
    time_intervals = []
    interval = 0
    for i in range(3):
        interval += time
        time_intervals.append(interval)
        
    # Builds isocrhone
    isochrone = client.isochrones(locations=to_travel, range_type='time',range=time_intervals, profile=transport)
    map_isochrone = folium.Map(location=[lat_start, lon_start], tiles='cartodbpositron', zoom_start=15)
    
    # Builds route between two locations
    route = client.directions(
        coordinates=to_travel,
        profile=transport,
        format='geojson',
        validate=False,
    )
    folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(map_isochrone)
    
    
    # GeoData from the isochrone
    folium.GeoJson(isochrone, name='isochrone').add_to(map_isochrone)
    
    interval_range = []
    for k in range(len(isochrone['features'])):
        interval = isochrone['features'][k]['properties']['value']/60
        interval_range.append(interval)

    # Places marker on the starting coordinates indicating the time range
    popup_message = f'Start Location. Outline shows areas {transport} within {interval_range} minutes'
    folium.Marker([lat_start, lon_start], popup=popup_message, tooltip='click').add_to(map_isochrone)
    
    popup_message_end = "End Location"  
    folium.Marker([lat_end, lon_end], popup=popup_message_end, tooltip='click').add_to(map_isochrone)

    # return map 
    return map_isochrone    

In [11]:
"""
Test code below returns an isochrone with starting coordinates of
University of Washington, and ending at the Seattle Space Needle
with intervals of of about 1 minute, and a transport
type of driving. 

Acceptable transport values are: driving-car, foot-walking, cycling-regular,
cycling-road, cycling-mountain, and cycling-electric
"""


start = [47.655548, -122.303200]
end = [47.620422, -122.349358]
# Time passed in seconds 
time = 100
# Mode of transport
transport = "driving-car"

generate_isochrone_dest(start, end, time, transport)